In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from warnings import simplefilter
simplefilter(action='ignore',category=FutureWarning)

In [2]:
#read the file
df = pd.read_csv('C:\\Users\\Ayomide Joe\\Documents\\UIC\\2021 Spring\\IDS 400\\bank.csv', delimiter=';')

In [44]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,999,0,unknown,0
1,33,employed,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,employed,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,employed,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,999,0,unknown,0
4,59,employed,married,secondary,0,0,1,0,unknown,5,may,226,1,999,0,unknown,0


In [4]:
df.shape

(4521, 17)

In [5]:
print(df.groupby('marital').size())

marital
divorced     528
married     2797
single      1196
dtype: int64


In [6]:
df = df.astype(str)

In [7]:
#convert selected columns that are catgorical into categorical variables
for col in ['job','marital','education', 'default', 'housing', 'loan', 'contact','month', 'poutcome', 'y']:
    df[col] = df[col].astype('category')

df.dtypes

age            object
job          category
marital      category
education    category
default      category
balance        object
housing      category
loan         category
contact      category
day            object
month        category
duration       object
campaign       object
pdays          object
previous       object
poutcome     category
y            category
dtype: object

In [8]:
#convert selected columns that are catgorical into integer variables
for col in ['age','balance','day', 'duration', 'campaign', 'pdays', 'previous']:
    df[col] = df[col].astype('int')

df.dtypes

age             int32
job          category
marital      category
education    category
default      category
balance         int32
housing      category
loan         category
contact      category
day             int32
month        category
duration        int32
campaign        int32
pdays           int32
previous        int32
poutcome     category
y            category
dtype: object

In [9]:
#encode variable as 1 & 0 
df['y'] = df['y'].cat.codes
df['default'] = df['default'].cat.codes


df['housing'] = df['housing'].cat.codes
df['loan'] = df['loan'].cat.codes

In [45]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,999,0,unknown,0
1,33,employed,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,employed,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,employed,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,999,0,unknown,0
4,59,employed,married,secondary,0,0,1,0,unknown,5,may,226,1,999,0,unknown,0
5,35,employed,single,tertiary,0,747,0,0,cellular,23,feb,141,2,176,3,failure,0
6,36,self-employed,married,tertiary,0,307,1,0,cellular,14,may,341,1,330,2,other,0
7,39,employed,married,secondary,0,147,1,0,cellular,6,may,151,2,999,0,unknown,0
8,41,self-employed,married,tertiary,0,221,1,0,unknown,14,may,57,2,999,0,unknown,0
9,43,employed,married,primary,0,-88,1,1,cellular,17,apr,313,1,147,2,failure,0


In [12]:
#re-categorize jobs into employed and self-employed
df2 = df

In [13]:
df2["job"] = df2["job"].apply(lambda x: x.replace("blue-collar", "employed"))
df2["job"] = df2["job"].apply(lambda x: x.replace("admin.", "employed"))
df2["job"] = df2["job"].apply(lambda x: x.replace("entrepreneur", "self-employed"))
df2["job"] = df2["job"].apply(lambda x: x.replace("housemaid", "employed"))
df2["job"] = df2["job"].apply(lambda x: x.replace("management", "employed"))
df2["job"] = df2["job"].apply(lambda x: x.replace("services", "employed"))
df2["job"] = df2["job"].apply(lambda x: x.replace("technician", "employed"))

In [14]:
print(df2.groupby('marital').size())
print(df2.groupby('education').size())
print(df2.groupby('job').size())
print(df2.groupby('contact').size())
print(df2.groupby('poutcome').size())

marital
divorced     528
married     2797
single      1196
dtype: int64
education
primary       678
secondary    2306
tertiary     1350
unknown       187
dtype: int64
job
employed         3690
retired           230
self-employed     351
student            84
unemployed        128
unknown            38
dtype: int64
contact
cellular     2896
telephone     301
unknown      1324
dtype: int64
poutcome
failure     490
other       197
success     129
unknown    3705
dtype: int64


In [47]:
df2.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,999,0,unknown,0
1,33,employed,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,employed,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,employed,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,999,0,unknown,0
4,59,employed,married,secondary,0,0,1,0,unknown,5,may,226,1,999,0,unknown,0


In [16]:
df2["pdays"].dtype

dtype('int32')

In [17]:
#replace -1 with 999
df2["pdays"].replace({-1: 999}, inplace=True)

In [49]:
df2.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,999,0,unknown,0
1,33,employed,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,employed,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,employed,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,999,0,unknown,0
4,59,employed,married,secondary,0,0,1,0,unknown,5,may,226,1,999,0,unknown,0


In [64]:
#create dummy variables 
to_dummy = pd.DataFrame(df2, columns=["job", "marital", "education", "poutcome" ])

In [65]:
dummy = pd.get_dummies(to_dummy, columns=["job", "marital", "education", "poutcome" ], prefix = ["job", "marital", "education", "outcome" ])
dummy.head()

,job_employed,job_retired,job_self-employed,job_student,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,outcome_failure,outcome_other,outcome_success,outcome_unknown
0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0
3,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
4,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1


In [66]:
#join dummies to dataframe
df3 = df2.join(dummy)

In [67]:
# Drop unnecccessary features and split into training/test sets
from sklearn.model_selection import train_test_split
features = df3.drop(['balance', 'contact', 'day', 'month','duration',"job", "marital", "education", "poutcome" ,'y'], axis=1)
labels = df3['y']

In [71]:
features.head()

,age,default,housing,loan,campaign,pdays,previous,job_employed,job_retired,job_self-employed,...,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,outcome_failure,outcome_other,outcome_success,outcome_unknown
0,30,0,0,0,1,999,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
1,33,0,1,1,1,339,4,1,0,0,...,1,0,0,1,0,0,1,0,0,0
2,35,0,1,0,1,330,1,1,0,0,...,0,1,0,0,1,0,1,0,0,0
3,30,0,1,1,4,999,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
4,59,0,1,0,1,999,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1


In [77]:
features.dtypes

age                    int32
default                 int8
housing                 int8
loan                    int8
campaign               int32
pdays                  int64
previous               int32
job_employed           uint8
job_retired            uint8
job_self-employed      uint8
job_student            uint8
job_unemployed         uint8
job_unknown            uint8
marital_divorced       uint8
marital_married        uint8
marital_single         uint8
education_primary      uint8
education_secondary    uint8
education_tertiary     uint8
education_unknown      uint8
outcome_failure        uint8
outcome_other          uint8
outcome_success        uint8
outcome_unknown        uint8
dtype: object

In [78]:
#split the data into test and train
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.4, random_state=0)

In [79]:
#Logistic regression model

scoring='accuracy'
seed=7
kfold=model_selection.KFold(n_splits=10,random_state=seed)
cv_results=model_selection.cross_val_score(LogisticRegression(),x_train,y_train,cv=kfold,scoring=scoring)
print(cv_results.mean(),cv_results.std())

0.8930649012372476 0.027462906728077358


In [80]:
logistic=LogisticRegression(C=1e5)
logistic.fit(x_train,y_train).score(x_val,y_val)
print(logistic.coef_)

[[ 2.19482965e-03  2.14012062e-01 -4.22466237e-01 -7.02279737e-01
  -1.26679287e-01 -5.37670850e-04  3.78023491e-02 -4.81874038e-01
   3.92070462e-01 -4.60288663e-01  3.80194654e-01 -4.93436061e-01
   3.81844075e-01  2.46291226e-02 -2.79997784e-01 -2.61209100e-02
  -7.52624265e-02  1.11427189e-01  4.85236315e-01 -8.02890648e-01
  -6.47122146e-01 -4.37596096e-01  1.44155413e+00 -6.38325462e-01]]


In [82]:
#models

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

models=[]
models.append(('LR',LogisticRegression()))
models.append(('LDA',LinearDiscriminantAnalysis()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('CART',DecisionTreeClassifier()))
models.append(('NB',GaussianNB()))
models.append(('SVM',SVC()))

results=[]
names=[]
for name,model in models:
    kfold=model_selection.KFold(n_splits=10,random_state=seed)
    cv_results=model_selection.cross_val_score(model,x_train,y_train,cv=kfold,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg="%s:%f (%f)" %(name,cv_results.mean(),cv_results.std())
    print(msg)

LR:0.893065 (0.027463)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\ProgramData\Anacond

LDA:0.892330 (0.025966)
KNN:0.879067 (0.030226)
CART:0.827818 (0.034213)
NB:0.804581 (0.024718)
SVM:0.890856 (0.027421)


In [ ]:
#best model is Logistic regression with mean of 0.892330